In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

import zmq
import numpy as np
from matplotlib import pyplot
import cv2

    
from PIL import Image
from skimage.io import imread, imsave
import io

In [2]:
model = load_model('generator_100.h5')

context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5555")

<SocketContext(bind='tcp://*:5555')>

In [3]:
# load an image
def load_image(pixels):
    # load image with the preferred size
    #pixels = load_img(filename, target_size=size)
    # convert to numpy array
    pixels = img_to_array(pixels)
    # scale from [0,255] to [-1,1]
    pixels = (pixels - 127.5) / 127.5
    # reshape to 1 sample
    pixels = np.expand_dims(pixels, 0)
    return pixels

In [ ]:
i = 0

while True:
    i+=1
    bytes_received = socket.recv()
    image = Image.open(io.BytesIO(bytes_received))
    image = load_image(image)
    gen_image = model.predict(image)
    # scale from [-1,1] to [0,1]
    gen_image = (gen_image + 1) / 2.0    
    print(i)

    imsave('predicted_map.png', (gen_image[0] * 255.0).astype(np.uint8))
    predicted_map = imread('predicted_map.png')
    predicted_map = np.rot90(predicted_map, 3)
    # predicted_map = predicted_map.ravel()
    #print(predicted_map.shape)
    bytes_to_send = predicted_map.tobytes()
    
    socket.send(bytes_to_send)

1
2


C:\Users\pawel\anaconda3\envs\mag1\lib\site-packages\ipykernel_launcher.py:13: UserWarning: predicted_map.png is a low contrast image
  del sys.path[0]


3
4
5
6
7
